## Uploading Data 

Uploading Categorical, Quantitative and Connectome Data

In [1]:
"""
ADHD Prediction Dataset - Data Merging Module

This notebook merges multiple ADHD-related datasets into a single comprehensive dataset
for machine learning analysis. The merged dataset combines neuroimaging connectome data,
behavioral questionnaires, demographic information, and target labels.


Project: ADHD Sex Prediction
Input Files: 4 separate datasets (connectome, quantitative, categorical, targets)
Output: Unified raw dataset for preprocessing pipeline
"""

import pandas as pd
import numpy as np
from pathlib import Path

# =============================================================================
# CONFIGURATION AND FILE PATHS
# =============================================================================

# Data file paths
DATA_DIR = Path("C:/Users/04ama/Downloads")
OUTPUT_DIR = Path(".")

# Input file paths
CONNECTOME_FILE = DATA_DIR / "TRAIN_FUNCTIONAL_CONNECTOME_MATRICES_new_36P_Pearson (1).csv"
QUANTITATIVE_FILE = DATA_DIR / "TRAIN_QUANTITATIVE_METADATA_new.xlsx"
CATEGORICAL_FILE = DATA_DIR / "TRAIN_CATEGORICAL_METADATA_new.xlsx"
TARGET_FILE = DATA_DIR / "TRAINING_SOLUTIONS (1).xlsx"

# Output file
OUTPUT_FILE = OUTPUT_DIR / "raw_dataset.csv"



In [2]:
# =============================================================================
# DATA LOADING FUNCTIONS
# =============================================================================

def load_dataset(file_path, dataset_name, file_type='csv'):
    """
    Load a dataset with error handling and basic validation.
    
    Parameters:
    -----------
    file_path : str or Path
        Path to the data file
    dataset_name : str
        Human-readable name for logging
    file_type : str
        File format ('csv' or 'excel')
    
    Returns:
    --------
    pd.DataFrame
        Loaded dataset
    
    Raises:
    -------
    FileNotFoundError
        If the file doesn't exist
    pd.errors.EmptyDataError
        If the file is empty
    """
    try:
        print(f"Loading {dataset_name} data")
        
        if file_type == 'csv':
            df = pd.read_csv(file_path)
        elif file_type == 'excel':
            df = pd.read_excel(file_path)
        else:
            raise ValueError(f"Unsupported file type: {file_type}")

        print(f"Successfully loaded: {df.shape[0]:,} rows × {df.shape[1]:,} columns")
        return df
        
    except FileNotFoundError:
        print(f" Error: File not found - {file_path}")
        raise
    except Exception as e:
        print(f"Error loading {dataset_name}: {str(e)}")
        raise



In [ ]:
# =============================================================================
# DATASET LOADING 
# =============================================================================

# Load Connectome Data (Functional brain connectivity matrices)
conn_data = load_dataset(CONNECTOME_FILE, "Connectome", "csv")


# Load Quantitative Metadata (Behavioral questionnaires and assessments)
quant_data = load_dataset(QUANTITATIVE_FILE, "Quantitative", "excel")


# Load Categorical Metadata (Demographics and categorical variables)
cat_data = load_dataset(CATEGORICAL_FILE, "Categorical", "excel")


# Load Target Data (ADHD outcome labels)
target_data = load_dataset(TARGET_FILE, "Target", "excel")


print(f"\n Dataset Summary:")

print(f"{'Connectome':<15} {conn_data.shape[0]:<8} {conn_data.shape[1]:<8} ")
print(f"{'Quantitative':<15} {quant_data.shape[0]:<8} {quant_data.shape[1]:<8} ")
print(f"{'Categorical':<15} {cat_data.shape[0]:<8} {cat_data.shape[1]:<8} ")
print(f"{'Target':<15} {target_data.shape[0]:<8} {target_data.shape[1]:<8} ")

Loading Connectome data
Successfully loaded: 1,213 rows × 19,901 columns
Loading Quantitative data
Successfully loaded: 1,213 rows × 19 columns
Loading Categorical data
Successfully loaded: 1,213 rows × 10 columns
Loading Target data
Successfully loaded: 1,213 rows × 3 columns

📈 Dataset Summary:
Connectome      1213     19901    
Quantitative    1213     19       
Categorical     1213     10       
Target          1213     3        


In [ ]:
# =============================================================================
# DATA MERGING PROCESS
# =============================================================================

def merge_datasets_sequentially(datasets, dataset_names, key='participant_id'):
    """
    Sequentially merge multiple datasets on a common key.
    
    Parameters:
    -----------
    datasets : list of pd.DataFrame
        List of datasets to merge
    dataset_names : list of str
        Names of datasets for logging
    merge_key : str
        Column name to merge on
    
    Returns:
    --------
    pd.DataFrame
        Final merged dataset
    
    Notes:
    ------
    Uses inner joins to ensure only participants with data in ALL datasets
    are included in the final merged dataset.
    """
  
    
    # Start with first dataset
    merged_df = datasets[0].copy()

    
    #  merge remaining datasets
    for i, (dataset, name) in enumerate(zip(datasets[1:], dataset_names[1:]), 1):
        before_shape = merged_df.shape
        
        # Perform inner join to keep only matching participant_ids
        merged_df = pd.merge(merged_df, dataset, on=key, how='inner')
        after_shape = merged_df.shape
        
        # Calculate merge statistics
        rows_lost = before_shape[0] - after_shape[0]
        cols_added = after_shape[1] - before_shape[1]
        
        print(f"   Step {i}: Added {name}")
        print(f"            • Rows: {before_shape[0]:,} → {after_shape[0]:,} ({-rows_lost:+,})")
        print(f"            • Cols: {before_shape[1]:,} → {after_shape[1]:,} ({cols_added:+,})")
        
        
    return merged_df

# Perform sequential merging
datasets = [conn_data, quant_data, cat_data, target_data]
dataset_names = ["Connectome", "Quantitative", "Categorical", "Target"]

merged_df = merge_datasets_sequentially(datasets, dataset_names)

print(f"Final dataset shape: {merged_df.shape[0]:,} rows × {merged_df.shape[1]:,} columns")

   Step 1: Added Quantitative
            • Rows: 1,213 → 1,213 (+0)
            • Cols: 19,901 → 19,919 (+18)
   Step 2: Added Categorical
            • Rows: 1,213 → 1,213 (+0)
            • Cols: 19,919 → 19,928 (+9)
   Step 3: Added Target
            • Rows: 1,213 → 1,213 (+0)
            • Cols: 19,928 → 19,930 (+2)

🎉 Merging Complete!
   Final dataset shape: 1,213 rows × 19,930 columns


In [ ]:
# =============================================================================
# DATA EXPORT AND SUMMARY
# =============================================================================

def save_merged_dataset(df, output_path):
    """
    Save merged dataset with metadata documentation.
    
    Parameters:
    -----------
    df : pd.DataFrame
        Merged dataset to save
    output_path : str or Path
        Output file path
    validation_report : dict
        Validation statistics for documentation
    """

    try:
        # Save main dataset
        df.to_csv(output_path, index=False)
        file_size_mb = Path(output_path).stat().st_size / (1024*1024)
        
        print(f"   ✅ Dataset saved: {output_path}")
        print(f"   📁 File size: {file_size_mb:.1f} MB")
        
        # Create metadata file
        metadata = {
            'creation_date': pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S'),
            'source_files': {
                'connectome': str(CONNECTOME_FILE),
                'quantitative': str(QUANTITATIVE_FILE),
                'categorical': str(CATEGORICAL_FILE),
                'target': str(TARGET_FILE)
            },
            'merge_method': 'inner_join',
            'merge_key': 'participant_id'
        }
        
        # Save metadata
        metadata_path = output_path.with_suffix('.json')
        import json
        with open(metadata_path, 'w') as f:
            json.dump(metadata, f, indent=2, default=str)
        
        
    except Exception as e:
        print(f"   ❌ Error saving dataset: {str(e)}")
        raise

# Save the merged dataset
save_merged_dataset(merged_df, OUTPUT_FILE)


print(f"📊 Final dataset: {merged_df.shape[0]:,} participants × {merged_df.shape[1]:,} features")
print(f"📁 Output file: {OUTPUT_FILE}")


   ✅ Dataset saved: raw_dataset.csv
   📁 File size: 446.5 MB

🎯 DATA MERGING PROCESS COMPLETE!
✅ Successfully created unified ADHD dataset
📊 Final dataset: 1,213 participants × 19,930 features
📁 Output file: raw_dataset.csv
🚀 Ready for preprocessing pipeline!


DONE